In [1]:
import opensim as osim
import os
import xml.etree.ElementTree as ET
from simFunctions import *
from subprocess import call
from stan_utils import subject_specific_isometric_force
from stan_utils import *
import pandas as pd
from ezc3d import c3d
from interpDFrame import *
from plot_utils import *
from matplotlib.backends.backend_pdf import PdfPages

# Walking Inverse Kinematics on one episode

## Create IK_setup_local.xml

In [5]:
model_name = 'scaled_model_P01.osim'

In [9]:
working_dir = 'P01/Python/'

In [11]:
output_dir = os.path.abspath(os.path.join(working_dir, 'walking', 'Walking16'))
trc_path = os.path.join(output_dir, 'task.trc')
trc_df = pd.read_csv(trc_path, sep='\t',skiprows=3, header=[0,1],index_col=0)

In [12]:
time_start = trc_df.loc[1, 'Time'][0]
time_stop=trc_df.loc[trc_df.shape[0], 'Time'][0]
time_range=f'{time_start} {time_stop}'

In [13]:
output_motion_file = os.path.join(output_dir, 'IK_results.mot')
# root_dir = os.path.abspath('..\\')

In [16]:
tree = ET.parse('IK_setup.xml')
root = tree.getroot()

In [17]:
for time in root.iter('time_range'):
    time.text = time_range

In [18]:
for output in root.iter('output_motion_file'):
    output.text = output_motion_file

In [19]:
for markers in root.iter('marker_file'):
    markers.text=trc_path

In [20]:
for model in root.iter('model_file'):
    model.text = os.path.join(model_name)

In [21]:
tree.write(os.path.join(output_dir, 'IK_setup_local.xml'))

## Run IK

In [22]:
# Inverse Kinematics
#os.chdir(baseDir + '/IK')
cmdprog = 'opensim-cmd'
cmdtool = 'run-tool'
cmdfile = os.path.join(output_dir, 'IK_setup_local.xml')
cmdfull = [cmdprog, cmdtool, cmdfile]
rc = runProgram(cmdfull)

[info] Preparing to run InverseKinematicsTool.

[info] Loaded model Rajagopal-generic-scaled from file scaled_model_P01.osim



               MODEL: Rajagopal-generic-scaled

         coordinates: 25

              forces: 83

           actuators: 83

             muscles: 80

            analyses: 0

              probes: 0

              bodies: 14

              joints: 14

         constraints: 2

             markers: 50

         controllers: 0

  contact geometries: 0

misc modelcomponents: 0

[info] Running tool IK_Rajagopal_scaled.

[info] Frame 0 (t = 0.0):	 total squared error = 0.00263900, marker error: RMS = 0.00937906, max = 0.0223558 (STRN)

[info] Frame 1 (t = 0.00500000):	 total squared error = 0.00258193, marker error: RMS = 0.00927708, max = 0.0222198 (STRN)

[info] Frame 2 (t = 0.01):	 total squared error = 0.00253139, marker error: RMS = 0.00918584, max = 0.0222585 (STRN)

[info] Frame 3 (t = 0.015):	 total squared error = 0.00250253, marker error: RMS = 0.009133

## Extract events, plot steps

#### record raw results into a pdf

In [23]:
# plot
plot_sto_file(os.path.join(output_dir, 'IK_results.mot'),
              os.path.join(output_dir, 'IK_results.pdf'), 3)

# Inverse Dynamics

## Create GRF_setup_local.xml

In [28]:
grf_file = os.path.join(output_dir, 'task_grf.mot')
tree = ET.parse('GRF_setup.xml')
root = tree.getroot()

In [29]:
n_of_plates = 5
first_foot = 'r'

In [30]:
if n_of_plates == 5 and first_foot == 'r':
    for file in root.iter('datafile'):
        file.text = grf_file

In [31]:
tree.write(os.path.join(output_dir, 'GRF_setup_local.xml'))

## Create ID_setup.xml

In [32]:
model_name = 'scaled_model_P01.osim'

In [33]:
path_to_data = output_dir
kinematics_path = os.path.join(path_to_data, 'IK_results.mot')
kinematics_df = pd.read_csv(kinematics_path, sep='\t',skiprows=10,index_col=0) #, header=[0,1]


In [34]:
time_start = kinematics_df.index[0]
time_stop=kinematics_df.index[-1]
time_range=f'{time_start} {time_stop}'

In [35]:
grf_setup = os.path.join(output_dir, 'GRF_setup_local.xml')

In [36]:
output_motion_file = os.path.join(output_dir, 'ID_results.mot')

In [37]:
tree = ET.parse('ID_setup.xml')
root = tree.getroot()

In [41]:
for time in root.iter('time_range'):
    time.text = time_range

for output in root.iter('results_directory'):
    output.text = output_dir

for file in root.iter('external_loads_file'):
    file.text = grf_setup

for coords in root.iter('coordinates_file'):
    coords.text=kinematics_path

for model in root.iter('model_file'):
    model.text = model_name

In [42]:
tree.write(os.path.join(output_dir, 'ID_setup_local.xml'))

## Run Inverse Dynamics

In [43]:
cmdprog = 'opensim-cmd'
cmdtool = 'run-tool'
cmdfile = os.path.join(output_dir, 'ID_setup_local.xml')
cmdfull = [cmdprog, cmdtool, cmdfile]
rc = runProgram(cmdfull)

[info] Preparing to run InverseDynamicsTool.

[info] Loaded model Rajagopal-generic-scaled from file scaled_model_P01.osim



               MODEL: Rajagopal-generic-scaled

         coordinates: 25

              forces: 83

           actuators: 83

             muscles: 80

            analyses: 0

              probes: 0

              bodies: 14

              joints: 14

         constraints: 2

             markers: 50

         controllers: 0

  contact geometries: 0

misc modelcomponents: 0

[info] Running tool ID_Rajagopal_scaled...

[warning] Couldn't find file 'r_pelvis.vtp'.

[warning] Couldn't find file 'l_pelvis.vtp'.

[warning] Couldn't find file 'sacrum.vtp'.

[warning] Couldn't find file 'r_femur.vtp'.

[warning] Couldn't find file 'r_tibia.vtp'.

[warning] Couldn't find file 'r_fibula.vtp'.

[warning] Couldn't find file 'r_patella.vtp'.

[warning] Couldn't find file 'r_talus.vtp'.

[warning] Couldn't find file 'r_foot.vtp'.

[warning] Couldn't find file 'r_bofoot.vtp

## Plot results

In [44]:
# plot
plot_sto_file(os.path.join(output_dir, 'ID_results.mot'),
              os.path.join(output_dir, 'ID_results.pdf'), 3)